# **Eventify**

An agent to automate event planning

- Human Input
- Async Execution
- Structered Data
- File output

### **Install Required Packages**

In [1]:
%pip install --quiet -U crewai
%pip install --quiet -U crewai_tools
%pip install --quiet -U pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### **Import Required Libraries**

In [3]:
from crewai import Agent, Task, Crew, LLM
from google.colab import userdata
import os
from IPython.display import Markdown
from IPython.display import  display
import textwrap

### **Set Up API Key**

In [4]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

### **Initialize an LLM**

In [5]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=1.0,
)

### **Define Tools**

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### **Create Agents**

Define Agents, and provide them a `role`, `goal` and `backstory`.

In [7]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm
)

In [8]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm
)

In [9]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm
)

### **Creating Venue Pydantic Object**

In [10]:
from pydantic import BaseModel

class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

### **Create Tasks**

In [11]:
# Task: Venue details task

venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    agent=venue_coordinator
)

In [12]:
# Task: Manage Logistics Task

logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [13]:
# Task: Marketing Task

marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    # async_execution=True,
    output_file="marketing_report.md",
    agent=marketing_communications_agent
)

### **Creating the Crew**

In [14]:
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True
)

### **Running the Crew and Displaying Response**

In [15]:
def to_markdown(text):
    # Replace bullet points with asterisks for markdown formatting
    text = text.replace("•", "  *")

    # Indent the text with a '>' character for blockquote style
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

In [19]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-12-15",
    'expected_participants': 100,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [20]:
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


# Agent: Venue Coordinator
## Thought: Thought: I need to find a venue in San Francisco suitable for a Tech Innovation Conference.  I should start by searching the internet for venues in San Francisco that can accommodate large events and have modern amenities.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"large event venues San Francisco tech conference\"}"
## Tool Output: 

Search results: Title: The 16 Best Conference Venues for Rent in San Francisco, CA
Link: https://www.tagvenue.com/us/hire/conference-venues/san-francisco
Snippet: Tagvenue offers an extensive range of conference venues in San Francisco, making it easy for you to find the perfect one. Select and book your venue today!
---
Title: The 25 Best Event Venues in San Francisco - Bizzabo
Link: https://www.bizzabo.com/blog/san-francisco-event-venues
Snippet: Featuring two levels,

ERROR:root:LiteLLM call failed: list index out of range


# Agent: Logistics Manager
## Task: Coordinate catering and equipment for an event with 100 participants on 2024-12-15.


# Agent: Logistics Manager
## Thought: Thought:I need to find catering services and equipment rental companies that can accommodate 100 people on December 15th, 2024.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"catering services for 100 people December 15 2024 [city/region]\"}"
## Tool Output: 

Search results: Title: 13 Best Wichita, KS Catering Companies | Expertise.com
Link: https://www.expertise.com/lifestyle/catering/kansas/wichita
Snippet: YaYa's Eurobistro's banquet areas can hold up to 100 seated guests. Buffet and plated lunch and dinner packages and hors d'oeuvres packages ...
---
Title: THE BEST 10 Caterers in WICHITA, KS - Yelp
Link: https://www.yelp.com/search?cflt=catering&find_loc=Wichita%2C+KS
Snippet: Best Caterers in Wichita, KS - Graze Craze - Northwest Wichita, Olive Tree Banquets & Catering, Gabby's Catering, Nora's 

ERROR:root:LiteLLM call failed: list index out of range


# Agent: Marketing and Communications Agent
## Task: Promote the Tech Innovation Conference aiming to engage at least100 potential attendees.


# Agent: Marketing and Communications Agent
## Thought: Thought: I need the location of the Tech Innovation Conference to proceed with marketing and securing catering and equipment rentals.  I'll need to find that information first.  Since I don't have it, I cannot complete the report yet.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Tech Innovation Conference 2024\"}"
## Tool Output: 

Search results: Title: Tech Conferences: The Best Tech Events Guide for 2024 - Bizzabo
Link: https://www.bizzabo.com/blog/technology-events
Snippet: This carefully curated conference directory features more than 100 tech conferences and events happening globally in 2024. Check it out!
---
Title: TechConnect World 2024
Link: https://www.techconnectworld.com/World2024/
Snippet: TechConnect World 2024 will take place June 17-19, 2024, Wa

In [22]:
Markdown("marketing_report.md")

I cannot complete the requested report.  The location (city and state/region) for the Tech Innovation Conference on December 15th, 2024, is required to confirm catering, equipment rentals, and to proceed with marketing and engagement activities.  Please provide the missing location details.

---